## 👨🏻‍💻 Stock Technical Analysis Framework

### 📥 Setups

Installation and Import of required packages

In [1]:
# !pip install -r requirements.txt

In [ ]:
import yfinance as yf
import backtrader as bt
import talib

import pandas as pd
import numpy as np

import plotly.graph_objects as go

Initialization of Analysis Parameters

In [3]:
TICKER: str = 'AAPL'

START_DATE: str = '2019-01-01'
END_DATE: str = '2025-07-25'

### 🏗️ Data Acquisition

Function for fetching stock OHCLV data

In [39]:
def fetch_price_data(ticker: str = TICKER, start: str = START_DATE, end: str = END_DATE) -> pd.DataFrame:
    """
    
    """
    
    data = yf.download(ticker, start=start, end=end, auto_adjust=True)
    data.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

    data['LogReturn'] = np.log(data['Close'] / data['Close'].shift(1))
    data.dropna(inplace=True)

    return data

The indicators will be split into 4 catgories, volume, volatility, momentum, and trend. The lists of specific indicators are as follows.
- Volume: Gauges <ins>strength of price movement</ins>.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Volume Rate of Change (VROC) |  |
| On Balance Volume (OBV) |  |
| Volume Weighted Average Price (VWAP) |  |
| Negative Volume Index (NVI) |  |
| Positive Volume Index (PVI) |  |
| Volume Oscillator (VolumeOsc) |  |
| Ease of Movement (EoM) |  |
| Force Index (ForceIndex) |  |
| Chaikin Money Flow (CMF)  |  |
| Accumulation/Distribution Line (ADLine) |  |

---
- Volatility: Measures the <ins>degree of price variation</ins> over time.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Historical Volatility (HistVolatility) |  |
| Chaikin Volatility (ChaikinVolatility) |  |
| Ulcer Index (UI) |  |
| Variance Indicator (Variance) |  |
| Average True Range (ATR) |  |
| Normalized ATR (NATR) |  |
| Bollinger Bands (BollingerBands) |  |
| Width of Bollinger Bands (BBWidth) |  |
| Donchian Channel (DonchianChannel) |  |
| Keltner Channel (KeltnerChannel) |  |

---
- Momentum: Measures the <ins>speed of price movement</ins>, useful for identifying overbought/oversold conditions.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Relative Strength Index (RSI) |  |
| Commodity Channel Index (CCI) |  |
| Money Flow Index (MFI) |  |
| Ultimate Oscillator (UltimateOscillator) |  |
| Chande Momentum Oscillator (CMO) |  |
| Stochastic Oscillator (Stochastic) |  |
| Stochastic RSI (StochRSI) |  |
| Rate of Change (ROC) |  |
| Williams %R (WilliamsR) |  |
| Momentum Indicator (Momentum) |  |

---
- Trend: Shows the <ins>direction and strength of price movement</ins>.

| Indicator                         |                                                                                       Description                                                                                                         |
|:---------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| Simple Moving Average (SMA) |  |
| Exponential Moving Average (EMA) |  |
| Double Exponential Moving Average (DEMA) |  |
| Weighted Moving Average (WMA) |  |
| Hull Moving Average (HMA) |  |
| Moving Average Convergence Divergence (MACD) |  |
| Detrended Price Oscillator (DPO) |  |
| Average Directional Index (ADX) |  |
| Aroon Indicator (AroonUp/AroonDown) |  |
| Ichimoku Cloud (Ichimoku) |  |

In [ ]:
class PandasData(bt.feeds.PandasData):
    lines = ('logreturn',)
    params = (('datetime', None), ('open', 'Open'), ('high', 'High'), ('low', 'Low'), ('close', 'Close'), ('volume', 'Volume'), ('logreturn', 'LogReturn'), ('openinterest', None))  # Parameters to convert to Backtrader datafeed

In [34]:
class MFI(bt.Indicator):
    lines = ('mfi',)
    params = dict(period=14)

    alias = ('MoneyFlowIndicator',)

    def __init__(self):
        tprice = (self.data.close + self.data.low + self.data.high) / 3.0
        mfraw = tprice * self.data.volume

        flowpos = bt.ind.SumN(mfraw * (tprice > tprice(-1)), period=self.p.period)
        flowneg = bt.ind.SumN(mfraw * (tprice < tprice(-1)), period=self.p.period)

        mfiratio = bt.ind.DivByZero(flowpos, flowneg, zero=100.0)
        self.l.mfi = 100.0 - 100.0 / (1.0 + mfiratio)

class DonchianChannels(bt.Indicator):
    '''
    Params Note:
      - `lookback` (default: -1)
        If `-1`, the bars to consider will start 1 bar in the past and the
        current high/low may break through the channel.
        If `0`, the current prices will be considered for the Donchian
        Channel. This means that the price will **NEVER** break through the
        upper/lower channel bands.
    '''

    alias = ('DCH', 'DonchianChannel',)

    lines = ('dcm', 'dch', 'dcl',)  # dc middle, dc high, dc low
    params = dict(
        period=20,
        lookback=-1,  # consider current bar or not
    )

    plotinfo = dict(subplot=False)  # plot along with data
    plotlines = dict(
        dcm=dict(ls='--'),  # dashed line
        dch=dict(_samecolor=True),  # use same color as prev line (dcm)
        dcl=dict(_samecolor=True),  # use same color as prev line (dch)
    )

    def __init__(self):
        hi, lo = self.data.high, self.data.low
        if self.p.lookback:  # move backwards as needed
            hi, lo = hi(self.p.lookback), lo(self.p.lookback)

        self.l.dch = bt.ind.Highest(hi, period=self.p.period)
        self.l.dcl = bt.ind.Lowest(lo, period=self.p.period)
        self.l.dcm = (self.l.dch + self.l.dcl) / 2.0  # avg of the above

In [ ]:
class IndicatorStrategy(bt.Strategy):
    # Initialize the indicators
    def __init__(self):
        # Trend
        self.sma = bt.ind.SMA(self.data.close, period=20)
        self.ema = bt.ind.EMA(self.data.close, period=20)
        self.dema = bt.ind.DEMA(self.data.close, period=20)
        self.wma = bt.ind.WMA(self.data.close, period=20)
        self.hma = bt.ind.HullMovingAverage(self.data.close, period=20)
        self.macd = bt.ind.MACD(self.data.close)
        self.dpo = bt.ind.DPO(self.data.close)
        self.adx = bt.ind.ADX(self.data)
        self.aroon = bt.ind.AroonIndicator(self.data, period=14)
        self.ichimoku = bt.ind.Ichimoku()
        
        # Momentum
        self.rsi = bt.ind.RSI(self.data.close, period=14)
        self.cci = bt.ind.CCI(self.data, period=20)
        self.mfi = MFI(self.data, period=14)
        self.uo = bt.ind.UltimateOscillator(self.data)
        self.cmo = talib.CMO(self.data.close, period=14)
        self.stoch = bt.ind.Stochastic(self.data)
        self.stoch_rsi = (self.rsi - bt.ind.Lowest(self.rsi, period=14)) / (bt.ind.Highest(self.rsi, period=14) - bt.ind.Lowest(self.rsi, period=14))
        self.roc = bt.ind.RateOfChange(self.data.close, period=14)
        self.williams_r = bt.ind.WilliamsR(self.data)
        self.momentum = bt.ind.Momentum(self.data.close, period=14)
        
        # Volatility
        ema_hl = bt.ind.EMA(self.data.high - self.data.low, period=10)
        drawdown = (self.data.close - bt.ind.Highest(self.data.close, period=14)) / bt.ind.Highest(self.data.close, period=14) * 100

        self.hist_vol = bt.ind.StdDev(self.data.logreturn, period=20) * (252 ** 0.5)
        self.chaikin_vol = (ema_hl - ema_hl(-10)) / ema_hl(-10)
        self.ui = bt.ind.StdDev(drawdown, period=14)
        self.variance = talib.VAR(self.data.close, period=20)
        self.atr = bt.ind.ATR(self.data, period=14)
        self.natr = (self.atr / self.data.close) * 100
        self.bbands = bt.ind.BollingerBands(self.data.close, period=20, devfactor=2)
        self.bbwidth = (self.bbands.top - self.bbands.bot) / self.bbands.mid * 100
        self.donchian = DonchianChannels(self.data, period=20)
        self.keltner_upper = self.ema + 2 * self.atr
        self.keltner_lower = self.ema - 2 * self.atr
        
        # Volume
        eom = ((self.data.high + self.data.low) / 2 - (self.data.high(-1) + self.data.low(-1)) / 2) * (self.data.high - self.data.low) / self.data.volume
        short_ema = bt.ind.EMA(self.data.volume, period=14)
        long_ema = bt.ind.EMA(self.data.volume, period=28)
        mfm = ((self.data.close - self.data.low) - (self.data.high - self.data.close)) / (self.data.high - self.data.low)

        self.vroc = bt.ind.RateOfChange(self.data.volume, period=14)
        self.obv = talib.OBV(self.data)
        # self.vwap = bt.ind.VWAP(self.data)
        # self.nvi = bt.ind.NVI(self.data)
        # self.pvi = bt.ind.PVI(self.data)
        self.volosc = 100 * (short_ema - long_ema) / long_ema
        self.eom = bt.ind.SMA(eom, period=14)  # smoothed EoM
        # self.force_index = bt.ind.ForceIndex(self.data)
        self.cmf = bt.ind.SumN(mfm * self.data.volume, period=20) / bt.ind.SumN(self.data.volume, period=20)
        self.ad = talib.AD(self.data)



In [44]:
data = fetch_price_data()
datafeed = PandasData(dataname=data)
cerebro = bt.Cerebro()
cerebro.adddata(datafeed)
cerebro.addstrategy(IndicatorStrategy)
cerebro.run()

[*********************100%***********************]  1 of 1 completed


AttributeError: module 'backtrader.talib' has no attribute 'CMO'

In [30]:
def plot(df: pd.DataFrame):
    fig = go.Figure(data=[go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'])])
    
    return fig

In [31]:
plot(data)